In [7]:
import os, random, sys, time, shutil, re, csv, datetime, pickle, math 

import pandas as pd

# from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.webdriver import Options

# from webdriver_manager.chrome import ChromeDriverManager

from random import seed
from random import random
from ..src.browser.
from bs4 import BeautifulSoup

ValueError: attempted relative import beyond top-level package

In [ ]:
# chrome_options = Options()
# #chrome_options.add_argument('--headless')
# chrome_options.add_argument('--no-sandbox')
# chrome_options.add_argument('--disable-dev-shm-usage')
# chrome_options.add_argument("--log-level=3")  # fatal
# chrome_options.add_argument("--start-maximized")

In [ ]:
# browser = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)

In [ ]:
browser = BrowserSetup()

In [ ]:
browser.get("https://www.linkedin.com/uas/login")

In [ ]:
# try:
#     f = open("../cookies/cookies.pkl")
#     f.close()

#     print('[Status] Cookies found! Logging in')

#     cookies = pickle.load(open("../cookies/cookies.pkl", "rb"))

#     for cookie in cookies:
#         browser.add_cookie(cookie)
# except IOError:
#     print('[Status] Cookies not found!')

#     print('[Status] Reading username and password file')

#     config = open('../config/config.txt')
#     lines = config.readlines()
#     config.close()
#     username = lines[0]
#     password = lines[1]

#     print('[Status] Entering username and password in their respective fields')

#     username_field = browser.find_element_by_id('username')
#     username_field.send_keys(username)

#     password_field = browser.find_element_by_id('password')
#     password_field.send_keys(password)

#     print('[Status] Submiting username and password...')

#     try:
#         password_field.submit()

#         print('[Status] Saving cookies...')
        
#         pickle.dump(browser.get_cookies(), open("../cookies/cookies.pkl","wb"))
        
#         print('[Status] Successfully saved cookies!')
#     except:
#         pass

In [ ]:
#pickle.dump(browser.get_cookies(), open("../cookies/cookies.pkl","wb"))

In [ ]:
# browser.current_url
# soup = BeautifulSoup(browser.page_source, features='html.parser')
# company_details = soup.find('dl').find_all()
# company_details_dict = {}
# for num, info in enumerate(company_details):
#     if(info.name == 'dt'):
#         company_details_dict[info.text.replace('\n', '').strip()] = company_details[num+1].text.replace('\n', '').strip()
# company_details_dict
#print(company_details_dict.get('Website', 'test'))
#print(company_details_dict.get('Websites', 'test'))

In [ ]:
seed(1)

In [ ]:
def get_employees_number(companies):
    
    #opening the csv to save the informations about the client
    try:
        df_info = pd.read_csv('companies_size.csv')
    except:
        df_info = pd.DataFrame()
    
    for processed, company in enumerate(companies):
        print("processed: " + str(processed))
        #searching for the company's linkedin page
        print("searching for the company " + company)
        #print(browser.current_url)
        browser.get("https://www.linkedin.com/search/results/companies/?keywords=" + company)
        time.sleep(2 + random())
        
        #getting company id
        soup = BeautifulSoup(browser.page_source, features='html.parser')
        code = soup.find_all('code')

        company_id = ''

        try:
            for code_str in code:
                if('urn:li:fsd_company:' in code_str.text):
                    company_id = re.findall(r"(?<=fsd_company:)\d+?(?=[,|\"])", code_str.text)[0]
                    #print(company_id)
                    if(company_id != '8175'):
                        break
            if company_id == '' or company_id == '8175':
                raise Exception("No Id for " + company)
        except:

            print("company " + company + " not found\n")
            print(browser.current_url)
            if "results/companies" not in browser.current_url: 
                print('logged out...')
                return
            info_to_append = {
                'company_searched': company, 
                'company_found': '', 
                'found': False, 
                'company_id': '', 
                'employees': '',
                'industry': '',
                'specialties': '',
                'headquarters': ''
                
            }
            df_info = df_info.append(info_to_append, ignore_index=True)
            df_info.to_csv("companies_size.csv", index=False)
            print('---------------------------------------------')
            continue 

        #accessing the company's page
        linkedin_url = "https://www.linkedin.com/company/" + company_id + "/about/"
        browser.get("https://www.linkedin.com/company/" + company_id + "/about/")
        time.sleep(2 + random())
        
        soup = BeautifulSoup(browser.page_source, features='html.parser')
        
        #searching for the company title
        try:
            company_title = soup.find(attrs={"class": "org-top-card-summary__title"})
            company_title = company_title.find("span").getText()
        except:
            try:
                time.sleep(1)
                company_title = soup.find(attrs={"class": "org-top-card-summary__title"})
                company_title = company_title.find("span").getText()
            except:
                company_title = ''
        
        #searching for the number of employees
        try:
            employees = soup.find(attrs={"class": "org-about-company-module__company-size-definition-text"}).getText().replace('\n', '').strip()
            print("employees:", employees)
        except:
            try:
                time.sleep(1)
                soup = BeautifulSoup(browser.page_source, features='html.parser')
                employees = soup.find(attrs={"class": "org-about-company-module__company-size-definition-text"}).getText().replace('\n', '').strip()
            except:
                employees = 'not found'
        
        #searching for the company info:
        company_details_dict = {}
        try:
            company_details = soup.find('dl').find_all()
            for num, info in enumerate(company_details):
                if(info.name == 'dt'):
                    company_details_dict[info.text.replace('\n', '').strip()] = company_details[num+1].text.replace('\n', '').strip()
            print("company details:", company_details_dict)
        except:
            print("Pass")
            pass
        
        
        browser.get('https://www.linkedin.com/jobs/search/?geoId=92000000&f_CR=103644278&f_F=it%2Ceng&f_C=' + company_id)
        time.sleep(1 + random())
        soup = BeautifulSoup(browser.page_source, features='html.parser')
        
        iteng_jobs = ''
        try:
            iteng_jobs = re.search(r"(.+\ )", soup.find("div", attrs={"class": "jobs-search-results-list__title-heading"}).find("small").text).group(1).strip()
        except:
            iteng_jobs = 'not found'
        
        
        #storing company data found
        print('-----------DADOS-----------')
        print('company_searched', company)
        print('company_found', company_title) 
        print('company_id', company_id)
        print('linkedin', linkedin_url)
        print('iteng_jobs', iteng_jobs)
        print('website', company_details_dict.get('Site', ''))
        print('employees', company_details_dict.get('Tamanho da empresa', ''))
        print('industry', company_details_dict.get('Setor', ''))
        print('specialties', company_details_dict.get('Especializações', ''))
        print('headquarters', company_details_dict.get('Sede', ''))
        
        info_to_append = {
            'company_searched': company,
            'company_found': company_title, 
            'found': True, 
            'company_id': company_id,
            'linkedin': linkedin_url,
            'it_eng_jobs': iteng_jobs,
            'website': company_details_dict.get('Site', ''),
            'employees': company_details_dict.get('Tamanho da empresa', ''),
            'industry': company_details_dict.get('Setor', ''),
            'specialties': company_details_dict.get('Especializações', ''),
            'headquarters': company_details_dict.get('Sede', '')
        }
        
        df_info = df_info.append(info_to_append, ignore_index=True)
        df_info.to_csv("companies_size.csv", index=False)
        print('---------------------------------------------')
  

In [ ]:
def get_company_searched():
    try:
        searched = pd.read_csv('companies_size.csv')['company_searched'].to_list()
        return list(filter(lambda elm: isinstance(elm, str), searched))
    except:
        return []
len(get_company_searched())

In [ ]:
def get_company_list():
    try:
        companies = pd.read_csv('Companies.csv')['Company'].to_list()
        return list(filter(lambda elm: isinstance(elm, str), companies))
    except:
        return []
len(get_company_list())

In [ ]:
companies = get_company_list()
companies_searched = get_company_searched()
to_search = [c for c in companies if c not in companies_searched]

while len(to_search) > 100:
    
    companies = get_company_list()
    companies_searched = get_company_searched()
    to_search = [c for c in companies if c not in companies_searched]
    
    count = int(20 + 50*random())
    print(count)
    
    print("Buscando total de:", len(to_search))
    get_employees_number(to_search[0:count])
    
    timeSleep = 30 + 60*random()
    print("Esperando um total de:", timeSleep)
    
    time.sleep(timeSleep)

In [ ]:
pd.read_csv('companies_size.csv')

In [ ]:
pd.read_csv('companies_size.csv')['found'].value_counts()